# 코사인 유사도

코사인 유사도를 사용하는 이유 : 비지도 KNN 으로 그룹화된 뉴스들을 수치화하여 비교하기 위해

Vectorizing 은 Tfidf 를 사용함.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# etoday_pre.csv 는 이미 전처리된 (형태소 분리된) 파일
NN_df = pd.read_csv("./etoday_pre.csv", names=["CNo", "Subjects", "Contents"])

tfidf = TfidfVectorizer()
# 전체 문서를 벡터화 시킴 = tfidf_matrix
tfidf_matrix = tfidf.fit_transform(NN_df['Contents'])

In [ ]:
def clac_sim(num):
    # target_matrix에는 문서 하나(이투데이)의 TF-IDF 값이 저장된다.
    target_matrix = tfidf.transform([NN_df._get_value(num, 'Contents')]).astype(np.float16)
    # cosin_matrix에는 전체 문서와 비교할 문서의 유사도값이 저장됨.
    cosine_matrix = cosine_similarity(target_matrix, tfidf_matrix)

# 보기 쉽게 맵핑하는 부분
    # news title과 id를 맵핑할 dictionary를 생성
    news2id = {}
    for i, c in enumerate(NN_df['CNo']):
        news2id[i] = c

    # id와 news title를 매핑할 dictionary를 생성
    id2news = {}
    for i, c in news2id.items():
        id2news[c] = i
# 맵핑 끝
    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[0])]
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    # sim_scores 에는 문서 번호(CNo) 와 유사도 값이 리스트 형태로 저장됨
    
    # sim_scores = [(news2id[i], round(score, 4)) for i, score in sim_scores if score >= 0.7 and score < 0.8]       # 70% 이상 80% 미만인 뉴스들을 출력
    sim_scores = [(news2id[i], round(score, 4)) for i, score in sim_scores]                                         # 제한 없이 출력

    return sim_scores

In [ ]:
# 확인.
clac_sim(60)

# 비지도 KNN

NearestNeighbors(n_neighbors=이웃갯수, radius=반경)



In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

# 사용 데이터 불러오기
NN_df = pd.read_csv("./etoday_pre.csv", names=["CNo", "Subjects", "Contents"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
# Tfidf로 토큰화
nn_tf = TfidfVectorizer()

nn_tf_matrix = nn_tf.fit_transform(NN_df['Contents'])
# 정규화
nn_tf_matrix = normalize(nn_tf_matrix)

In [ ]:
# 이웃 검색을 위한 비지도 학습기 객체 형성
# n_neighbors = 이웃수
# radius = 반경

neigh = NearestNeighbors(n_neighbors=10, radius=1.0)

neigh.fit(nn_tf_matrix)

cluster를 위한 새로운 label 'cluster' 생성

In [ ]:
NN_df["cluster"] = 0

In [ ]:
# 확인.
NN_df.head()

## 1) 근접 이웃 개수 이용

kneighbors

In [ ]:
# kneighbors : 점의 k-이웃을 찾는다.
# 
nei_list = neigh.kneighbors(nn_tf_matrix[1004])
print(nei_list[1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# me 에 기준점(클러스터 중심)을 저장
me = nei_list[1][0][0]

for n, i in enumerate(nei_list[1][0]):
    # 기준점과 이웃되는 점의 코사인 유사도를 계산
    cosine_matrix = cosine_similarity(nn_tf_matrix[me], nn_tf_matrix[i])
    print('거리 : {: .4f} / 제목 : {} '.format(nei_list[0][0][n], NN_df["Subjects"][i]))
    print("코사인 유사도 : ", round(cosine_matrix[0][0], 4))

## 2) 거리(반경) 기반 cluster

radius_neigbors

test 용 파일. 사용 함수는 아래에 있다.

In [ ]:
# test 파일!!!
from sklearn.metrics.pairwise import cosine_similarity
# 점의 지정된 반경 내에서 이웃을 찾습니다.
nbrs = neigh.radius_neighbors(nn_tf_matrix[100], sort_results = True)     # nn_tf_matrix[index] : index 값으로 중심이 되는 뉴스 선택
# print(nbrs)           # 확인용  # nbr[0][0]에 거리, nbr[1][0]에 index가 들어간다.
me = nbrs[1][0][0]      # 중심 뉴스의 인덱스 = me

for n, i in enumerate(nbrs[1][0]):
    cosine_matrix = cosine_similarity(nn_tf_matrix[me], nn_tf_matrix[i])
    if (n == 0 or (cosine_matrix[0][0] >=0.6 and cosine_matrix[0][0]<0.8)):
        print('거리 : {: .4f} / 제목 : {} '.format(nbrs[0][0][n], NN_df["Subjects"][i]))
        print("index : ", i)
        print("유사도 : ", round(cosine_matrix[0][0], 4))
        print("-"*50)

### 함수화

거리 내에 있는 모든 원소를 비교하고 싶기 때문에, 2) 거리 기반 cluster 로 함수화 진행

In [ ]:
# 위의 코드를 함수화 시키자
# index : 뉴스의 index 번호, 중심이 될 뉴스의 index를 넣어라.
# tfidf_matrix : 우리가 계산한 tfidf_matrix

def clustering(index, tfidf_matrix):
    nbrs = neigh.radius_neighbors(tfidf_matrix[index], sort_results = True)

    me = nbrs[1][0][0]
    clu_list = []

    for n, i in enumerate(nbrs[1][0]):
        cosine_matrix = cosine_similarity(tfidf_matrix[me], tfidf_matrix[i])
        if (n == 0 or (cosine_matrix[0][0] >=0.6 and cosine_matrix[0][0]<0.8)):
            clu_list.append(i)
    return clu_list

### 동작부분
clustering 동작 부분.

In [ ]:
# 함수를 사용해 clustering 시작
cnt = 0     # 묶인 cluster 에 이름대신 숫자를 붙인다.
for line in range(len(NN_df)):
    # 해당 line이 클러스터가 형성되지 않았을 경우(클러스터 넘버가 0일 경우)에 함수 동작
    if NN_df["cluster"][line] == 0:
        cnt += 1
        clu_list = clustering(line, nn_tf_matrix)
    for i in clu_list:
        NN_df["cluster"][i] = cnt   # 클러스터 숫자를 붙여준다.
    print(NN_df["cluster"][i])

결과 확인 작업 및 저장

In [ ]:
# 출력 왼쪽이 클러스터 이름, 오른쪽이 클러스터에 소속된 뉴스 수
NN_df["cluster"].value_counts()

In [ ]:

for i in range(len(NN_df)):
    if NN_df["cluster"][i] == 5337:     # 5337 부분에 클러스터 이름을 넣으면, 해당 클러스터의 속한 뉴스 index 출력
        print(i)

In [ ]:
# 확인용
NN_df.head()

In [ ]:
# 저장용
NN_df.to_csv("./clustering_etoday.csv", index=False)

### 시각화

큰 의미는 없었다.

In [ ]:
# 시각화, 의미 없다.
import matplotlib.pyplot as plt
plt.figure(figsize=(48,5))

plt.title('Cluster' , fontsize=20)

plt.ylabel('member count' , fontsize=15)
plt.xlabel('cluster' , fontsize=15)

NN_df['cluster'].value_counts().value_counts().plot.bar()

plt.show()

# 워드 랭크로 주요단어 추출해보기

생각보다 비슷한 뉴스가 없었나 보다.

각각 클러스터된 문장들이 어떤 주제로 엮었는지 알아보자.

그러기 위해서는 각 문장의 핵심 단어, 키워드를 추출할 필요가 있어 보인다!

### test 1

In [ ]:
import pandas as pd
from textrank import KeywordSummarizer

# 
def go(sent):
    return sent

clu_df = pd.read_csv("./clustering_etoday.csv")


keyword_extractor = KeywordSummarizer(
    tokenize = go,
    window = 2,
    verbose = True
)
# keyword_extractor = summarize(
#     tokenize = None,
#     min_sim = 0.5,
#     verbose = True
# )

# keyword_extractor = summarizer.KeywordSummarizer(tokenize=go, window = 2)
keywords  = keyword_extractor.summarize(clu_df["Contents"][0], topk=10)
print(keywords)

## TextRank 테스트 코드 작성

In [ ]:
import pandas as pd
from textrank import KeywordSummarizer          # 구현된 textrank
from konlpy.tag import Komoran                  # Komoran를 사용하기 위한 import -> 이미 전처리가 되어있다면 사용하지 않아도 된다.

clu_df = pd.read_csv("./clustering_etoday.csv")

In [ ]:
komoran = Komoran()             # 코모란 객체 선언
def komoran_tokenize(sent):     # 코모란 사용 함수
    words = komoran.pos(sent, join=True)
    return words

In [ ]:
# text_rank 테스트 코드
# df 전체가 돌아간다. 
# 마지막에 CSV 파일이 저장된다.

import re

clu_df["textrank_output"] = ""

keyword_extractor = KeywordSummarizer(
    tokenize= komoran_tokenize,         # 위에서 만든 코모란 함수를 넣어준다.
    window= -1,
    verbose =False
)

# 전처리
for idx in range(len(clu_df)):      # 데이터의 수(index)만큼 돈다
    word = clu_df['Contents'][idx].replace('\n', '').replace('  ', '')    
    result = re.sub('[(-=.\'·ㆍ>▷▶◆…‘’“”\"#/?:$})]', '', word)          # 특수문자 제거
    clu_df['textrank_output'][idx] = [str(result)]
    print(0,idx)

for idx in range(len(clu_df)):   
    outputs = []
    try:
        sents = clu_df['textrank_output'][idx]
        keywords = keyword_extractor.summarize(sents, topk=10)
        for word, rank in keywords:
            # outputs.append('{} ({:.3})'.format(word, rank))       # 결과 + 점수
            word = word.split("/")[0]
            outputs.append('{}'.format(word))                       # 결과만
        print(idx)
        outputs = " ".join(outputs)
        clu_df['textrank_output'][idx] = outputs
        
    except:
        continue
    
clu_df.to_csv('TextRank_Data.csv', encoding='utf-8-sig', mode = 'a')    # 저장

In [ ]:
# 결과 및 점수가 저장됨.
keywords

In [ ]:
text_df = pd.read_csv("./TextRank_Data.csv")
text_df.head()

In [ ]:
# clustering 결과로 CNo 랑 Subject 출력하기.
for line in range(len(clu_df)):
    if clu_df["cluster"][line] == 4636:
        print(clu_df["CNo"][line], clu_df["Subjects"][line])

In [ ]:
key_W = []
for line in range(len(clu_df)):
    if text_df["cluster"][line] == 4636:
        print(text_df["CNo"][line], text_df["textrank_output"][line])
        test =  text_df["textrank_output"][line].split()
        key_W += test
print(key_W)

In [ ]:
count={}
for i in key_W:
    try: 
        count[i] += 1
    except: 
        count[i]=1
pgm_lang_val_reverse = sorted(count.items(), reverse=True, key=lambda item: item[1])

print(pgm_lang_val_reverse)


## TextRank 함수화

In [ ]:
import pandas as pd
from textrank import KeywordSummarizer
from konlpy.tag import Komoran

clu_df = pd.read_csv("./clustering_etoday.csv")

In [ ]:
komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    return words

In [ ]:
# 나는 이미 전처리가 되어있으므로, split으로 나눠주기만 함.
def go(sent):
    words = sent.split(" ")
    return words

In [ ]:
import re
# contents에는 리스트 형태의 기사 내용이 들어가면 된다. 각 인덱스에 한 기사내용이 들어간다.
# 한줄짜리 contents도 가능
def text_rank(contents):
    textrank_output = []
    all_out = []
    keyword_extractor = KeywordSummarizer(
        # tokenize= komoran_tokenize,   # 전처리가 안되어 있을경우
        tokenize= go,                   # 전처리가 되어있을경우
        window= -1,
        verbose =False
    )
    # 전처리 : contents의 수만큼 전처리 한 후, textrank_output에 저장
    for idx in range(len(contents)):
        word = contents[idx].replace('\n', '').replace('  ', '')
        result = re.sub('[(-=.\'·ㆍ>▷▶◆…‘’“”\"#/?:$})]', '', word)
        textrank_output.append(str(result))
    for idx in range(len(textrank_output)):   
        outputs = []
        try:
            sents = textrank_output[idx]
            keywords = keyword_extractor.summarize([sents], topk=10)        # summarize 에는 sents가 list의 형태로 들어가기 때문에, []를 씌워준다.    
            for word, rank in keywords:
                word = word.split("/")[0]                           # 단어만 추출(단어/형태소분류)
                outputs.append('{}/{:.3}'.format(word, rank))       # 점수를 3자리까지만 표시
                # outputs.append('{}'.format(word))
            
            outputs = " ".join(outputs)
            all_out.append(outputs)                                 # all_out에 저장
        except:
            print("except")
            continue
        
    return all_out

# 각 cluster에서 5위 안에 드는 단어 뽑기

텍스트랭크로 출력된 결과를 모두 더해 해당 단어의 갯수만큼 나눠서 내림차순으로 정렬.

In [ ]:
# 위에서 분류된 파일을 불러온다.
import pandas as pd
clu_df = pd.read_csv("./clustering_etoday.csv")

In [ ]:
# value_counts() : 해당 column 에 속한 원소 각각의 갯수를 출력함
clu_df['cluster'].value_counts()[:20]

In [ ]:
clu_df['cluster'].value_counts()[20:50]

In [ ]:
# clustering 결과로 CNo 랑 Subject 출력하기.
cluster_list = []
for line in range(len(clu_df)):
    if clu_df["cluster"][line] == 5041:
        print(clu_df["CNo"][line], clu_df["Subjects"][line])
        cluster_list.append(clu_df["Contents"][line])
        
score_sum = {}      # 단어 스코어 합산 측정
count = {}          # 단어 갯수 측정
for news in text_rank(cluster_list):
    word_score = news.split(" ")
    
    for i in word_score:
        word, score = i.split("/")
        try: 
            score_sum[word] += float(score)
            count[word] += 1
        except: 
            score_sum[word] = float(score)
            count[word] = 1

result = {}
for word in score_sum:
    result[word] = score_sum[word]/count[word]      # 단어 스코어 합산 결과에서 갯수를 나눠줬다.
pgm_lang_val_reverse = sorted(result.items(), reverse=True, key=lambda item: item[1])       # 점수(items)로 내림차순 정렬

print(pgm_lang_val_reverse[:5])